In [ ]:
import pandas as pd

# 读取CSV文件
df = pd.read_csv('../data/cleaned_emails.csv')

In [ ]:
print(df.info())
print("-------------------------------------")
print(df.describe())
print("-------------------------------------")
print(df.head())

In [ ]:
from neo4j import GraphDatabase

uri = "bolt://localhost:7687"
username = "neo4j"
password = ""

driver = GraphDatabase.driver(uri, auth=(username, password))

def test_connection():
    with driver.session() as session:
        result = session.run("RETURN 'Hello, World!' AS message")
        print(result.single()["message"])

test_connection()

In [ ]:
# 函数定义：导入数据到Neo4j
def load_data_to_neo4j(driver, df):
    with driver.session() as session:
        for index, row in df.iterrows():
            # 创建Email节点及其与Sender和Recipient的关系
            session.run("""
            MERGE (sender:Sender {name: $sender})
            MERGE (recipient:Recipient {name: $recipient})
            CREATE (email:Email {id: $message_id, subject: $subject, text: $text, date: $date})
            MERGE (sender)-[:SENT]->(email)
            MERGE (email)-[:TO]->(recipient)
            """, parameters={
                'sender': row['sender'],
                'recipient': row['recipient1'],
                'message_id': row['Message-ID'],
                'subject': row['subject'] if pd.notna(row['subject']) else "No Subject",
                'text': row['text'],
                'date': row['date']
            })

# 调用函数，导入数据
load_data_to_neo4j(driver, df)

In [ ]:
# 关闭驱动程序
driver.close()